<a href="https://colab.research.google.com/github/zmbq/butbul-halacha-ingest/blob/main/research/butbul_halacha_find_categories.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load all the data from the database, prepare for analysis

In [1]:
import os
from sqlalchemy import create_engine
from google.colab import userdata

# Retrieve the database URL from Colab secrets
DATABASE_URL = userdata.get('POSTGRES_URL')

# Create a database engine
engine = create_engine(DATABASE_URL)

print("Database connection established successfully!")

Database connection established successfully!


In [3]:
from sqlalchemy import inspect

inspector = inspect(engine)

# Get table names
table_names = inspector.get_table_names()
print("Tables in the database:")
for table_name in table_names:
    print(f"- {table_name}")

print("\nColumns in each table:")
# Get column names for each table
for table_name in table_names:
    print(f"\nTable: {table_name}")
    columns = inspector.get_columns(table_name)
    for column in columns:
        print(f"- {column['name']} ({column['type']})")

Tables in the database:
- videos
- alembic_version
- video_metadata
- transcripts
- transcription_segments
- transcription_chunks
- embeddings
- embeddings_cache
- taggings
- tags

Columns in each table:

Table: videos
- video_id (VARCHAR(20))
- url (VARCHAR(255))
- title (VARCHAR(255))
- description (TEXT)
- published_at (TIMESTAMP)
- created_at (TIMESTAMP)
- updated_at (TIMESTAMP)
- duration_seconds (INTEGER)

Table: alembic_version
- version_num (VARCHAR(32))

Table: video_metadata
- video_id (VARCHAR(20))
- hebrew_date (VARCHAR(50))
- subject (VARCHAR(500))
- created_at (TIMESTAMP)
- updated_at (TIMESTAMP)
- day_of_week (VARCHAR(20))

Table: transcripts
- video_id (VARCHAR(20))
- source (VARCHAR(20))
- language (VARCHAR(10))
- full_text (TEXT)
- segments (JSONB)
- created_at (TIMESTAMP)
- updated_at (TIMESTAMP)

Table: transcription_segments
- id (INTEGER)
- video_id (VARCHAR(20))
- source (VARCHAR(20))
- segment_index (INTEGER)
- start (DOUBLE PRECISION)
- duration (DOUBLE PRECISI

/tmp/ipython-input-614922236.py:15: SAWarning: Did not recognize type 'vector' of column 'vector'
  columns = inspector.get_columns(table_name)


- id (INTEGER)
- text (TEXT)
- model (VARCHAR(128))
- vector (NULL)
- created_at (TIMESTAMP)
- updated_at (TIMESTAMP)

Table: taggings
- id (INTEGER)
- tag_id (INTEGER)
- video_id (VARCHAR(20))
- source (VARCHAR(100))
- created_at (TIMESTAMP)
- updated_at (TIMESTAMP)

Table: tags
- id (INTEGER)
- name (VARCHAR(20))
- description (TEXT)
- type (VARCHAR(30))
- created_at (TIMESTAMP)
- updated_at (TIMESTAMP)


In [15]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text

# Define the VideoData class
class VideoData:
    def __init__(self, video_id, subject, gregorian_date, hebrew_date, subject_embedding, first_chunk_text, first_chunk_embedding):
        self.video_id = video_id
        self.subject = subject
        self.gregorian_date = gregorian_date
        self.hebrew_date = hebrew_date
        self.subject_embedding = subject_embedding
        self.first_chunk_text = first_chunk_text
        self.first_chunk_embedding = first_chunk_embedding

    def __repr__(self):
        return f"VideoData(video_id={self.video_id}, subject='{self.subject}', gregorian_date='{self.gregorian_date}', hebrew_date='{self.hebrew_date}', first_chunk_text='{self.first_chunk_text[:50]}...')"

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

# Query the database to get the required video data
video_data_list = []
try:
    query = text("""
        SELECT
            vm.video_id,
            vm.subject,
            v.published_at AS gregorian_date,
            vm.hebrew_date,
            sec.vector AS subject_embedding,
            tc.text AS first_chunk_text,
            tcec.vector AS first_chunk_embedding
        FROM video_metadata vm
        JOIN videos v ON vm.video_id = v.video_id
        JOIN embeddings se ON v.video_id = se.video_id AND se.transcription_chunk_id IS NULL
        JOIN embeddings_cache sec ON se.source_cache_id = sec.id
        JOIN (
            SELECT
                video_id,
                MIN(first_segment_id) as min_first_segment_id
            FROM transcription_chunks
            GROUP BY video_id
        ) as min_tc ON vm.video_id = min_tc.video_id
        JOIN transcription_chunks tc ON vm.video_id = tc.video_id AND tc.first_segment_id = min_tc.min_first_segment_id
        JOIN embeddings tce ON tc.id = tce.transcription_chunk_id AND tce.video_id = vm.video_id
        JOIN embeddings_cache tcec ON tce.source_cache_id = tcec.id
    """)

    result = session.execute(query)

    for row in result:
        video_data = VideoData(
            video_id=row.video_id,
            subject=row.subject,
            gregorian_date=row.gregorian_date,
            hebrew_date=row.hebrew_date,
            subject_embedding=row.subject_embedding,
            first_chunk_text=row.first_chunk_text,
            first_chunk_embedding=row.first_chunk_embedding
        )
        video_data_list.append(video_data)

    print(f"Successfully loaded {len(video_data_list)} videos.")
    # Display the first few entries to verify
    for i in range(min(5, len(video_data_list))):
        print(video_data_list[i])

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    session.close()

Successfully loaded 1011 videos.
VideoData(video_id=_03R7gJHoKM, subject='אכילת פיתה שחוממה ע"ג סיר בשרי עם מאכל חלבי', gregorian_date='2022-12-14 21:35:45+00:00', hebrew_date='כ"ה כסלו התשפ"ג', first_chunk_text='שלום רב אחד מבני הבית רצה לחמם פיתה על גבי הסיר שנ...')
VideoData(video_id=_19uanvcg30, subject='מהי ברכת הריח על ציפורן וקינמון?', gregorian_date='2025-05-03 21:31:16+00:00', hebrew_date='ו' אייר התשפ"ה', first_chunk_text='‫שלום רב. יום ראשון, וב ואייר. ‫מהי ברכת הריח שמבר...')
VideoData(video_id=_AlfP4rDA1k, subject='דבר תורה לפרשת 'וישלח'', gregorian_date='2023-11-30 22:11:51+00:00', hebrew_date='י"ח כסלו התשפ"ד', first_chunk_text='‫שלום רב. יום שישי יותחת כסלב, ‫ערב שבת קודש פרשת ...')
VideoData(video_id=_d1Ag4AvoSY, subject='אדם שטעה ושתה בתענית האם יברך ברכה אחרונה?', gregorian_date='2022-07-24 09:38:48+00:00', hebrew_date='ו' אב התשפ"ב', first_chunk_text='שלום רב נשאלתי שאלה מעניינת בתענית האחרונה של יהוד...')
VideoData(video_id=_dwr1xhkVTE, subject='לברך ברכות השחר בבו

Now that the video data is loaded, we need to extract the embedding vectors. We will create two NumPy arrays: one for the subject embeddings and one for the first chunk embeddings.

In [16]:
import numpy as np

subject_embeddings = []
first_chunk_embeddings = []
video_ids = [] # Store video_ids to link embeddings back to videos

for video_data in video_data_list:
    # Since we used JOINs, we expect these to be non-None
    subject_embeddings.append(video_data.subject_embedding)
    video_ids.append(video_data.video_id) # Append video_id for each subject embedding
    first_chunk_embeddings.append(video_data.first_chunk_embedding)


# Convert the lists to NumPy arrays
subject_embeddings_array = np.array(subject_embeddings)
first_chunk_embeddings_array = np.array(first_chunk_embeddings)
video_ids_array = np.array(video_ids)


print(f"Extracted {len(subject_embeddings)} subject embeddings.")
print(f"Subject embeddings array shape: {subject_embeddings_array.shape}")
print(f"Extracted {len(first_chunk_embeddings)} first chunk embeddings.")
print(f"First chunk embeddings array shape: {first_chunk_embeddings_array.shape}")

Extracted 1011 subject embeddings.
Subject embeddings array shape: (1011,)
Extracted 1011 first chunk embeddings.
First chunk embeddings array shape: (1011,)
